ENTRAINEMENT DU MODELE DE FORET ALEATOIRE (RANDOM FOREST)

Importation des librairies

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly_express as px
import numpy as nu
from sklearn.model_selection import train_test_split # Pour le split train/test
from sklearn.ensemble import RandomForestClassifier # Modèle sélectionné (catégorisation/regression)
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report # Evaluation des résultats

train = pd.read_csv('data/train.csv')

Nettoyage des données

In [12]:
# Extraction titre
train['Title'] = train['Name'].str.extract(r',\s*([^\.]*)\s*\.', expand=False)
train['Title'] = train['Title'].str.strip()
# Verification des entrées concernées
train[train['Age'].isnull()].groupby(['Title', 'Pclass', 'Sex', 'Parch']).size()
# Conversion données NULL 
age_medians = train.groupby(['Title', 'Pclass'])['Age'].median()
# Fonction pour imputer/remplacer les âges manquants en fonction des catégories concernées
def impute_age(row):
    if pd.isnull(row['Age']):
        return age_medians.get((row['Title'], row['Pclass']), train['Age'].median())
    return row['Age']
# Application des changements à la table
train['Age'] = train.apply(impute_age, axis=1)
# Conversion du type de données de la colonne 'Sex' en valeurs numériques
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Sex'] = le.fit_transform(train['Sex'])

# Conversion du type de données de la colonne 'Title' & création d'une table de correspondance pour comprendre les modifications
train['Title'] = le.fit_transform(train['Title'])
title_mapping = pd.DataFrame({
    'Title': le.classes_,
    'Encoded': le.transform(le.classes_)
})
title_mapping.sort_values('Encoded')

,Title,Encoded
0,Capt,0
1,Col,1
2,Don,2
3,Dr,3
4,Jonkheer,4
5,Lady,5
6,Major,6
7,Master,7
8,Miss,8
9,Mlle,9


MODELE 2 - Random Forest 

Sélection des features (X) et de la target (y)

In [13]:
features = ['Age', 'Sex', 'Pclass', 'Title']
X = train[features]

y = train['Survived']

Séparation des données d'entraînement et de test

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Entraînement du modèle Random Forest

In [15]:
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=1
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

Evaluation

In [16]:
print('Accuracy : ', accuracy_score(y_test, y_pred)) # Précision
print('Matrice de confusion : ',confusion_matrix(y_test, y_pred)) # Croisement entre les vraies données et celles prédites
print('Rapport de classification : ', classification_report(y_test, y_pred))

Accuracy :  0.7653631284916201
Matrice de confusion :  [[87 19]
 [23 50]]
Rapport de classification :                precision    recall  f1-score   support

           0       0.79      0.82      0.81       106
           1       0.72      0.68      0.70        73

    accuracy                           0.77       179
   macro avg       0.76      0.75      0.75       179
weighted avg       0.76      0.77      0.76       179



Amélioration du modèle </br>
=> Recherche de nouveaux hyperparamètres pour le modèle Random Forest
=> Utilisation de la technique de cross-validation pour trouver des paramètres adéquates

In [17]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300], #nb d'arbres dans la forêt
    'max_depth': [None, 5, 10], # profondeur max des arbres
    'min_samples_split': [2, 4, 10] # nb mini. d'échantillons requis pour diviser un noeud
}

grid = GridSearchCV(RandomForestClassifier(),
                    param_grid, #appel du dictionnaire avec les paramètres à tester
                    cv=5, # validation croisée à 5 plis (division du dataset en 5 parties, 4 d'entrainement, 1 de test)
                    scoring='accuracy')

grid.fit(X_train, y_train) 

print('Meilleurs paramètres : ', grid.best_params_)
print('Score CV : ', grid.best_score_)


Meilleurs paramètres :  {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
Score CV :  0.8173840244262779


Résumé </br>
Nombre d'arbres : 100 </br>
La profondeur max. des arbres (max_depth) : 5 </br>
Nombre d'échantillons requis pour la division des noeuds : 2 </br>


Test des nouveaux hyper paramètres

In [20]:
model_opti = RandomForestClassifier(
    n_estimators= 100,
    max_depth= 5,
    min_samples_split=2,
    random_state=1,
)

model_opti.fit(X_train, y_train)

y_pred_opti = model_opti.predict(X_test)

print('Accuracy : ', accuracy_score(y_test, y_pred_opti)) # Précision
print('Matrice de confusion : ',confusion_matrix(y_test, y_pred_opti)) # Croisement entre les vraies données et celles prédites
print('Rapport de classification : ', classification_report(y_test, y_pred_opti))

Accuracy :  0.7597765363128491
Matrice de confusion :  [[93 13]
 [30 43]]
Rapport de classification :                precision    recall  f1-score   support

           0       0.76      0.88      0.81       106
           1       0.77      0.59      0.67        73

    accuracy                           0.76       179
   macro avg       0.76      0.73      0.74       179
weighted avg       0.76      0.76      0.75       179



Ca a diminué le score... 